### 1. 安装依赖

In [1]:
!pip install langchain
!pip install langchain-community
!pip install tiktoken
!pip install transformers
!pip install docarray

### 2.导入模型

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings

### 3. 加载模型

In [3]:
model_local = ChatOllama(model="qwen:7b")

### 4. 读取文件并分词

In [5]:
documents = TextLoader("C:\\Users\\Administrator\\Desktop\\三国演义.txt",encoding='utf-8').load()

In [6]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)

In [7]:
doc_splits = text_splitter.split_documents(documents)

In [8]:
print(doc_splits)

[Document(page_content='第一回\u3000宴桃园豪杰三结义\u3000斩黄巾英雄首立功\n\n话说天下大势，分久必合，合久必分。周末七国分争，并入于秦；及秦灭之后，楚汉分争，又并入于汉。汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国。推其致乱之由，始于桓、灵二帝。桓帝禁锢善类，崇信宦官。及桓帝崩，灵帝即位，大将军窦武、太傅陈蕃共相辅佐。时有宦官曹节等弄权，窦武、陈蕃谋诛之，机事不密，反为所害：中涓自此愈横。\n\n建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起，只见一条大青蛇从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温德殿中。秋七月，有虹见于玉堂，五原山岸尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由。议郎蔡邕上疏，以为霓堕鸡化，乃妇寺干政之所致，言颇切直。帝览奏叹息，因起更衣。曹节在后窃视，悉宣告左右，遂以他事陷邕于罪，放归田里。后张让、赵忠、封谞、段珪、曹节、侯览、蹇硕、程旷、夏恽、郭胜十人朋比为奸，号为“十常侍”。帝尊信张让，呼为阿父。朝政日非，以致天下人心思乱，盗贼蜂起。\n\n时钜鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤角至一洞中，以天书三卷授之，曰：“此名《太平要术》，汝得之，当代天宣化，普救世人。若萌异心，必获恶报。”角拜问姓名，老人曰：“吾乃南华老仙也。”言讫，化阵清风而去。角得此书，晓夜攻习，能呼风唤雨，号为“太平道人”。\n中平元年正月内，疫气流行，张角散施符水，为人治病，自称“大贤良师”。\n角有徒弟五百余人，云游四方，皆能书符念咒。次后徒众日多，角乃立三十六方，大方万余人，小方六七千，各立渠帅，称为将军；讹言“苍天已死，黄天当立”，又云“岁在甲子，天下大吉”，令人各以白土书“甲子”二字于家中大门上。\n青、幽、徐、冀、荆、扬、兖、豫八州之人，家家侍奉大贤良师张角名字。角遣其党马元义暗赍金帛，结交中涓封谞以为内应。角与二弟商议曰：“至难得者民心也。今民心已顺，若不乘势取天下，诚为可惜。”遂一面私造黄旗，约期举事；一面使弟子唐周驰书

### 5. 嵌入并存储

In [9]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')

In [10]:
vectorstore = DocArrayInMemorySearch.from_documents(doc_splits, embeddings)

In [11]:
retriever = vectorstore.as_retriever()

### 6. 向模型提问

In [12]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

In [13]:
prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [15]:
print(chain.invoke("身长七尺，细眼长髯的是谁？"))

身长七尺，细眼长髯的人物是曹操，字孟德，沛国谯郡人。

